#**Google Analytics (Universal Analytics) Reporting API**

Этот скрипт запрашивает UA отчет у Google Analytics Reporting API и возвращает его в виде pandas dataframe.

Последовательность действий:

1. Создать проект в Google Cloud Platform в Google-аккаунте, который относится к вертикали клиента Риалвеб
https://cloud.google.com/resource-manager/docs/creating-managing-projects



2.  Активировать возможность использования Google Analytics Reporting API для этого проекта в Google Cloud Platform: кнопка "+ENABLE APIS AND SERVICES" в разделе APIs & Services, поиск и переход в Google Analytics Reporting API, кнопка "Enable" 
https://developers.google.com/analytics/devguides/reporting/core/v4


3. Создать сервисный аккаунт, связанный с ранее созданным проектом (кнопка "+CREATE CREDENTIALS" в разделе "CREDENTIALS", выбрать поле "Create service account), выглядит как адрес алектронной почты, например,   ga-api-client@aqueous-thought-372224.iam.gserviceaccount.com)

4. Создать ключ API  для сервисного аккаунта в виде JSON-файла
https://cloud.google.com/iam/docs/creating-managing-service-account-keys

5. В панели управления "Администратор" кабинета Google Analytics в настройках аккаунта или ресурса дайте вашему сервисному аккаунту доступ "Читатель" (в случае если кабинет принадлежит клиенту Риалвеб, выдачу доступа "Читатель" для сервисного аккаунта необходимо запросить у клиента, предоставив ему имя сервисного аккаунта)

6. Используя набор метрик (metrics) и измерений (dimensions), выполнить запрос в соответствии с кодом ниже. Список возможных метрик и измерений: https://ga-dev-tools.appspot.com/dimensions-metrics-explorer/

Примечание: если сервисный аккаунт и проект клиента Риалвеб с активированным Google Analytics Reporting API уже были созданы в рамках Google Cloud Platform вертикали, то начинать следует с создания собственного ключа API (п.4)

Гайды Google Developers:

https://developers.google.com/analytics/devguides/reporting/data/v1/rest/v1beta/properties 

https://googleapis.dev/python/analyticsdata/latest/data_v1beta/beta_analytics_data.html

###**Инициализация библиотек и параметров подключения**

In [ ]:
#Объявляем необходимые библиотеки

import numpy as np
import pandas as pd
from google.oauth2 import service_account
from apiclient.discovery import build

In [ ]:
#Подключаем GDrive и указываем путь к папке с ключем API 

from google.colab import drive
drive.mount('/content/drive')
ga_keys = '/content/drive/MyDrive/Colab_Notebooks/glowing-box-371708-1be164ea4b4e.json'

Mounted at /content/drive


ВАЖНО: не публикуйте и не передавайте третьим лицам ваш ключ API, в противном случае возможен несанкционированный доступ к данным, содержащимся в кабинете Google Analitycs, со стороны третьих лиц

In [ ]:
#Указываем номер представления Google Analitycs, к которому хотим подключиться

your_view_id = '269579389'


###**Блок объявления функций для получения отчета Google Analytics**

In [ ]:
#Объявляем набор функций необходимых для получения отчета в формате pandas dataframe

def format_summary(response):
    try:
        #create row index
        try: 
            row_index_names = response['reports'][0]['columnHeader']['dimensions']
            row_index = [ element['dimensions'] for element in response['reports'][0]['data']['rows'] ]
            row_index_named = pd.MultiIndex.from_arrays(np.transpose(np.array(row_index)), 
                                                        names = np.array(row_index_names))
        except:
            row_index_named = None
        
        #extract column names
        summary_column_names = [item['name'] for item in response['reports'][0]
                                ['columnHeader']['metricHeader']['metricHeaderEntries']]
    
        #extract table values
        summary_values = [element['metrics'][0]['values'] for element in response['reports'][0]['data']['rows']]
    
        #combine. I used type 'float' because default is object, and as far as I know, all values are numeric
        df = pd.DataFrame(data = np.array(summary_values), 
                          index = row_index_named, 
                          columns = summary_column_names).astype('float')
    
    except:
        df = pd.DataFrame()
        
    return df

def format_pivot(response):
    try:
        #extract table values
        pivot_values = [item['metrics'][0]['pivotValueRegions'][0]['values'] for item in response['reports'][0]
                        ['data']['rows']]
        
        #create column index
        top_header = [item['dimensionValues'] for item in response['reports'][0]
                      ['columnHeader']['metricHeader']['pivotHeaders'][0]['pivotHeaderEntries']]
        column_metrics = [item['metric']['name'] for item in response['reports'][0]
                          ['columnHeader']['metricHeader']['pivotHeaders'][0]['pivotHeaderEntries']]
        array = np.concatenate((np.array(top_header),
                                np.array(column_metrics).reshape((len(column_metrics),1))), 
                               axis = 1)
        column_index = pd.MultiIndex.from_arrays(np.transpose(array))
        
        #create row index
        try:
            row_index_names = response['reports'][0]['columnHeader']['dimensions']
            row_index = [ element['dimensions'] for element in response['reports'][0]['data']['rows'] ]
            row_index_named = pd.MultiIndex.from_arrays(np.transpose(np.array(row_index)), 
                                                        names = np.array(row_index_names))
        except: 
            row_index_named = None
        #combine into a dataframe
        df = pd.DataFrame(data = np.array(pivot_values), 
                          index = row_index_named, 
                          columns = column_index).astype('float')
    except:
        df = pd.DataFrame()
    return df

def format_report(response):
    summary = format_summary(response)
    pivot = format_pivot(response)
    if pivot.columns.nlevels == 2:
        summary.columns = [['']*len(summary.columns), summary.columns]
    
    return(pd.concat([summary, pivot], axis = 1))

def run_report(body, credentials_file):
    #Create service credentials
    credentials = service_account.Credentials.from_service_account_file(credentials_file, 
                                scopes = ['https://www.googleapis.com/auth/analytics.readonly'])
    #Create a service object
    service = build('analyticsreporting', 'v4', credentials=credentials)
    
    #Get GA data
    response = service.reports().batchGet(body=body).execute()
    
    return(format_report(response))

###**Получение отчета Google Analytics**

Отчет может содержать аггрегированную часть и (или) pivot-часть, для этого нужно выбрать один из предложенных ниже шаблонов формы отчета, в которой необходимо указать метрики (metrics), измерения (dimensions) и временные рамки.

Метрики и измерения: https://ga-dev-tools.appspot.com/dimensions-metrics-explorer/

In [ ]:
#Шаблон для получения аггрегированного отчета по указанным метрикам и измерениям (наиболее часто используемый)

summary_body = {'reportRequests': [{'viewId': your_view_id, 
                            'dateRanges': [{'startDate': '2022-06-01', 'endDate': '2022-08-01'}],
                            'metrics': [{'expression': 'ga:sessions'}, 
                                        {'expression': 'ga:totalEvents'}, 
                                        {"expression": "ga:avgSessionDuration"}],
                            'dimensions': [{'name': 'ga:country'}],
                          }]}

In [ ]:
#Шаблон для получения аггрегированного отчета совмещенного с pivot-отчетом по указанным метрикам и измерениям

body = {'reportRequests': [{'viewId': your_view_id, 
                            'dateRanges': [{'startDate': '2022-06-01', 'endDate': '2022-08-01'}],
                            'metrics': [{'expression': 'ga:users'}, 
                                        {"expression": "ga:bounceRate"}],
                            'dimensions': [{'name': 'ga:dayMonth'}],
                            "pivots": [{"dimensions": [{"name": "ga:channelGrouping"}],
                                        "metrics": [{"expression": "ga:users"},
                                                    {"expression": "ga:bounceRate"}]
                                       }]
                          }]}

In [ ]:
#Шаблон для получения pivot-отчета по указанным метрикам и измерениям (в том числе внешним измерениям)

pivot_body = {'reportRequests': [{'viewId': your_view_id, 
                            'dateRanges': [{'startDate': '2022-06-01', 'endDate': '2022-08-01'}],
                            'dimensions': [{'name':  "ga:channelGrouping"}],
                            "pivots": [{"dimensions": [{"name": 'ga:yearMonth'}],
                                        "metrics": [{"expression": "ga:users"},
                                                    {"expression": "ga:newUsers"},
                                                    {"expression": "ga:timeOnPage"}]
                                       }]
                          }]}

In [ ]:
#Укороченный шаблон для получения отчета по указанным метрикам без группировок и в рамках последней недели

short_body = {  "reportRequests":
  [{
      "viewId": your_view_id,
      "dateRanges": [{"startDate": "2022-06-01", "endDate": "2022-08-01"}], # or 7daysAgo or yesterday
      "metrics": [{"expression": "ga:users"}]
    }]}

In [ ]:
#Шаблон для получения укороченного pivot-отчета без внешних группировок

untidy_body = {'reportRequests': [{'viewId': your_view_id, 
                            'dateRanges': [{'startDate': '2022-06-01', 'endDate': '2022-08-01'}],
                            "pivots": [{"dimensions": [{"name": 'ga:yearMonth'}, {"name": "ga:channelGrouping"}],
                                        "metrics": [{"expression": "ga:users"},
                                                    {"expression": "ga:timeOnPage"}]
                                       }]
                          }]}

Пример выведенного отчета (выбрали шаблон **summary_body**)

In [ ]:
ga_report = run_report(summary_body, ga_keys)
ga_report 

,ga:sessions,ga:totalEvents,ga:avgSessionDuration
ga:country,,,
Russia,32.0,118.0,1520.625
